## Роевой интеллект: частицы

Реализую алгоритм роя частиц (Particle Swarm Optimization, PSO) —
метод оптимизации, вдохновлённый коллективным поведением стай птиц и косяков рыб.
В отличие от муравьиного алгоритма, где агенты обмениваются информацией
через феромоны, здесь частицы (агенты) взаимодействуют друг с другом
напрямую, обновляя свои скорости и позиции на основе как собственного опыта,
так и успехов соседей.

Каждая частица помнит лучшее решение, которое она нашла,
а также знает лучшее решение в своём окружении (или во всей популяции,
в зависимости от варианта алгоритма). Практическая задача — минимизация сложной
математической функции с множеством локальных минимумов. Частицы «летают»
по пространству решений, со временем сближаясь к глобальному минимуму.

Для визуализации я использовал двухмерную функцию с рельефной поверхностью,
по которой можно наблюдать, как частицы постепенно сходятся к наиболее
выгодной точке. Я также визуализировал траектории движения частиц, чтобы
отследить, насколько сильно на них влияют собственные открытия и находки других.

In [50]:
%use kandy

In [51]:
data class Particle(
  var position: MutableList<Double>,
  var velocity: MutableList<Double>,
  var bestPosition: MutableList<Double>,
  var bestFitness: Double
)

In [52]:
fun rastrigin(position: List<Double>): Double {
  return position.size + position.sumOf { x -> x * x - 5_000 * cos(x / 2.25) }
}

plot {
  line {
    val keys = (-100..100).toList()

    x(keys)
    y(keys.map { rastrigin(listOf(it.toDouble())) })
  }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="OrgWl6"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[-100.0,-99.0,-98.0,-97.0,-96.0,-95.0,-94.0,-93.0,-92.0,-91.0,-90.0,-89.0,-88.0,-87.0,-86.0,-85.0,-84.0,-83.0,-82.0,-81.0,-80.0,-79.0,-78.0,-77.0,-76.0,-75.0,-74.0,-73.0,-72.0,-71.0,-70.0,-69.0,-68.0,-67.0,-66.0,-65.0,-64.0,-63.0,-62.0,-61.0,-60.0,-59.0,-58.0,-57.0,-56.0,-55.0,-54.0,-53.0,-52.0,-51.0,-50.0,-49.0,-48.0,-47.0,-46.0,-45.0,-44.0,-43.0,-42.0,-41.0,-40.0,-39.0,-38.0,-37.0,-36.0,-35.0,-34.0,-33.0,-32.0,-31.0,-30.0,-29.0,-28.0,-27.0,-26.0,-25.0,-24.0,-23.0,-22.0,-21.0,-20.0,-19.0,-18.0,-17.0,-16.0,-15.0,-14.0,-13.0,-12.0,-11.0,-10.0,-9.0,-8.0,-7.0,-6.0,-5.0,-4.0,-3.0,-2.0,-1.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0],
"y":[5525.514273612494,4802.783456761544,5053.403720154746,6190.402150611489,7954.970572267207,9966.752146288978,11797.74495618415,13055.4630632048,13459.196783790445,12894.554766712148,11435.69030826131,9330.89331774395,6954.347423912866,4733.42584845996,3065.6525064273183,2241.462846638853,2387.772051934161,3443.315335216024,5170.551728096623,7201.818448137024,9110.768020603848,10495.207874745984,11055.246626254311,10651.563171545778,9332.476870320212,7324.5590620178555,4988.610104738519,2749.5556270720676,1013.8831974674486,90.6615208802059,131.4881850179545,1101.0341487999885,2783.9081984017594,4826.5076025352955,6805.723356969751,8311.150543847794,9024.830278618869,8783.029331567832,7608.0536026225345,5703.914070247291,3416.6873234185114,1167.2726515321915,-630.3889658588187,-1649.7438651038528,-1715.0752912457883,-835.6452797076263,796.1049633150151,2841.946514475688,4883.5808367480895,6503.915461783588,7368.105332327293,7288.613418269665,6261.649285611751,4467.905580297729,2237.4493276190674,-14.410309066959826,-1867.8168734142555,-2979.9450031994884,-3151.6116249063843,-2365.9773566262006,-791.818411098172,1249.266734034476,3345.344133097864,5074.183410436597,6085.297401616924,6168.042041247233,5292.544644178757,3615.508187984276,1449.7638207908321,-796.5124376710173,-2699.108566620418,-3900.2012203877475,-4177.881041100469,-3489.269793662461,-1978.8516351878125,49.60025227124777,2192.0469918768995,4022.689032959037,5176.699285284133,5421.109263941289,4700.07502001831,3145.7276568039733,1052.5001690430677,-1180.0810826458205,-3125.024607838169,-4410.838515254876,-4793.711632898777,-4204.886180001661,-2764.017449295853,-755.9247045103975,1424.7493909174145,3350.21810431806,4642.669878835238,5047.677368763108,4483.632841065207,3057.604611869177,1044.533601840789,-1166.187866514947,-3146.3752546147257,-4512.248346794493,-4999.0,-4512.248346794493,-3146.3752546147257,-1166.187866514947,1044.533601840789,3057.604611869177,4483.632841065207,5047.677368763108,4642.669878835238,3350.21810431806,1424.7493909174145,-755.9247045103975,-2764.017449295853,-4204.886180001661,-4793.711632898777,-4410.838515254876,-3125.024607838169,-1180.0810826458205,1052.5001690430677,3145.7276568039733,4700.07502001831,5421.109263941289,5176.699285284133,4022.

In [53]:
// --- Параметры задачи ---
val dimensions = 2
val particleCount = 30
val iterations = 100
val lowerBound = -5.12
val upperBound = 5.12

// --- Параметры PSO ---
val inertiaWeight = 0.7
val cognitiveComponent = 1.5  // личный опыт (c1)
val socialComponent = 1.5     // коллективный опыт (c2)

In [54]:
// --- Инициализация роя ---
val swarm = List(particleCount) {
  val position = List(dimensions) { Random.nextDouble(lowerBound, upperBound) }.toMutableList()
  val velocity = List(dimensions) { Random.nextDouble(-1.0, 1.0) }.toMutableList()
  val fitness = rastrigin(position)
  Particle(position, velocity, position.toMutableList(), fitness)
}

plot {
  points {
    x(swarm.map { it.position[0] })
    y(swarm.map { it.position[1] })
  }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="RkNprw"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[2.831534356897917,0.5113800504304562,0.004271921012603208,-2.4280561276139645,-5.004302052337784,-2.3265466487450226,-2.081876535658739,2.2188412870172343,2.886965313396126,-3.3358677853979124,1.96547561132288,-4.3019531862009845,-0.3371943622357856,-2.7342856931725543,-4.608997332225453,2.4247633693900887,-1.117761965879562,0.6747958310588551,-4.2112658310546545,-4.185453970381955,-2.3145535316941857,2.2596969818762727,-0.41559946570306217,-3.603296462102518,3.2632329006525103,-3.3967028616999038,0.9207669792927264,-3.198647874587108,-2.3973467671399282,-1.831632757643015],
"y":[-4.314791631657929,3.4309215291443484,1.4488111875803442,-0.251989832720402,-4.390898684899185,-2.2362448990204156,2.011393255986677,4.487621309199377,-4.801398743169969,-1.7370045299698926,0.18900112816667924,-3.694153638209851,3.2199003528347605,-1.2588591255407757,3.793102448681471,-1.891467097739827,-0.45379770281123566,-0.120448423915815,-0.7756222523317442,-3.7697852127061333,-1.5363055980942706,-3.677226782679063,-3.0200624782034504,-1.9303662376046713,-3.1203900520222634,3.386892074067492,-0.1994498070883246,-0.7182376545060274,2.0712691349519483,2.21631483830875]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"point",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"float",
"column":"y"
}]
}
}],
"spec_id":"92"
};
 var containerDiv = document.getElementById("RkNprw");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5 
 
 
 
 
 
 
 
 
 -4 
 
 
 
 
 
 
 
 
 -3 
 
 
 
 
 
 
 
 
 -2 
 
 
 
 
 
 
 
 
 -1 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 -5 
 
 
 
 
 
 
 -4 
 
 
 
 
 
 
 -3 
 
 
 
 
 
 
 -2 
 
 
 
 
 
 
 -1 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 y 
 
 
 
 
 x

In [55]:
import kotlin.math.*

// --- Лучшее глобальное решение ---
var globalBest = swarm.minBy { it.bestFitness }.bestPosition.toMutableList()
var globalBestFitness = rastrigin(globalBest)

// --- Лог для визуализации ---
val trajectoryLog = mutableListOf<Triple<Int, Double, Double>>() // (iter, x, y)

for (iter in 0 until iterations) {
  for (particle in swarm) {
    // Обновить скорость и позицию
    for (d in 0 until dimensions) {
      val r1 = Math.random()
      val r2 = Math.random()
      val cognitive = cognitiveComponent * r1 * (particle.bestPosition[d] - particle.position[d])
      val social = socialComponent * r2 * (globalBest[d] - particle.position[d])
      particle.velocity[d] = inertiaWeight * particle.velocity[d] + cognitive + social
      particle.position[d] += particle.velocity[d]
      particle.position[d] = particle.position[d].coerceIn(lowerBound, upperBound)
    }

    val fitness = rastrigin(particle.position)
    if (fitness < particle.bestFitness) {
      particle.bestFitness = fitness
      particle.bestPosition = particle.position.toMutableList()
    }

    if (fitness < globalBestFitness) {
      globalBestFitness = fitness
      globalBest = particle.position.toMutableList()
    }

    // Логировать движение (только первую частицу, для примера)
    if (particle == swarm[0]) {
      trajectoryLog.add(Triple(iter, particle.position[0], particle.position[1]))
    }
  }
}

In [56]:
import org.jetbrains.kotlinx.dataframe.api.dataFrameOf

val x = trajectoryLog.map { it.second }
val y = trajectoryLog.map { it.third }
val step = trajectoryLog.map { it.first.toDouble() }

val data = dataFrameOf("x" to x, "y" to y)

plot(data) {
  path {
    x("x")
    y("y")
    width = 2.0
    color = Color.hex("#4682B4")
  }

  points {
    x("x")
    y("y")
    size = 4.0
    color = Color.RED
  }

  x.axis.name = "X"
  y.axis.name = "Y"
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="65g1yZ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"x":[0.27667811018092525,-1.5329992695631127,-2.24971359814245,-1.1511848918358067,1.2007215574202288,-0.7546037848025196,-1.245481580301033,-0.575081018536361,0.5235434594655556,0.17821369508834978,-0.9331116636105399,-0.8833104127136358,0.25674940453476824,0.6372917326443648,-1.0786407214784781,-1.0747935129017545,-0.16178085881644022,0.37164534259360604,0.5154022257699655,-0.14669989977358866,-0.48368462057220707,-0.35349041914269796,0.25458514838213586,0.5769176247447995,-0.07962430696458356,-0.6181373933471626,-0.7355864755987663,0.0374452439331433,0.24927899402798132,-0.10716828379065474,-0.3337054671127284,0.16177913805108657,0.34517986099030223,-0.4073828371922584,-0.38840816246715015,0.14394508957974483,0.11765054114497807,0.016937994493612246,0.05708012950995845,0.07894280174664645,0.04442397515064539,0.06781639126763629,0.07974346738951749,0.05819290084571011,0.002014225584169936,-0.004683926266933038,-0.008264301915921417,-0.0038734822571558115,0.0024817749039140625,0.001338232660434971,-0.008784368102825084,-0.007576894677091298,9.085397921302718E-4,-0.005033936841403152,-0.008954719222824967,-0.003874604348971857,0.0016217635155366195,-0.007714940294454958,-0.007505021101153613,-0.001587291939947149,0.001689044059579918,0.0022117895898440742,0.002011916123915304,-8.01715520404654E-5,-4.4279669533789454E-4,0.0012772283629962759,6.023694951308709E-4,2.083478016144717E-4,0.0011568817004956736,1.6113817607109663E-4,2.0903964135140824E-4,0.0011617924839963192,9.830283004926108E-4,-1.2123326354558585E-4,-8.885825324144812E-4,-3.110323028236036E-4,1.4964171145703077E-4,1.364768260752122E-4,1.2936632495088923E-6,3.856931105893387E-5,1.725018170950519E-4,1.899396113667833E-4,-5.786153254881514E-5,-9.415710624043007E-5,4.8762953826085055E-5,1.7739527381491167E-4,1.0381881131848843E-4,-2.4080181316374458E-5,1.3235132106194863E-4,5.3774179673346355E-5,1.1456870294079248E-5,8.39379663695388E-5,3.19843888176889E-5,7.486722632015537E-5,1.0307064778059669E-4,9.462060617674626E-5,3.3960823471752924E-5,2.8438761936091058E-5,6.063337251272602E-5,6.558794184644148E-5],
"y":[-3.820266198337518,-0.28543895108823625,2.4390251098559386,-0.623006930729749,-1.8586975152215897,-0.27339592679690905,0.9615751412340752,0.00815485473361166,-0.6022670536360114,0.707710138543899,0.522416407394166,-0.28758335875411367,-0.5507587275853527,0.6936041924390677,1.3722377054373105,-0.03417731184738515,-0.951720929831561,0.29543867823192604,0.8580749895078186,-0.10728462161012853,-0.7209770294305761,-1.054367256780441,-0.9694393038549081,0.37872722027594685,0.3672235029850476,0.18036040682675702,-0.36394652086990664,-0.35774441681789626,0.024394659502476657,0.18264255658487594,-0.28921082374534135,-0.27135653812982485,0.2219498027649437,0.13489256106815592,-0.3081163811066179,-0.42627151491042753,-0.035652597701904654,0.25018588109483636,-0.21893555268032244,-0.3257288197762262,-0.35796808886460524,0.19858052103999285,0.46548563640540874,-0.1229380243356123,-0.39779586724249794,-0.07294604850618236,0.18335389658929846,-0.15416631081571464,-0.18984340243438613,-0.07935939171623393,0.031067083961582406,0.08490507873097608,-0.05615795989944811,-0.048132022773426764,0.1216699182830816,-0.04014041273752961,-0.10410786203212657,0.06526728910606636,0.0749886623576521,-0.052281334746318645,-0.011820717526892476,0.03412962086008086,-0.02126140687308572,-0.029184213084396313,-0.03190672684429757,-0.010355211907012472,0.01347072794035533,0.01642531893118007,-0.013889630346962313,-0.026615732945444205,-0.02388523354194682,-0.00465255